# TA-Lib Candlestick Pattern Scanner (Swing Trading)

This notebook uses **TA-Lib** to scan your tickers for **classic candlestick patterns** relevant to swing trading on **Daily** or **4-Hour** timeframes.

**Patterns covered**
- **Bullish:** Bullish Engulfing, Morning Star, Hammer, Piercing Line, Three White Soldiers, Bullish Harami
- **Bearish:** Bearish Engulfing, Evening Star, Shooting Star, Dark Cloud Cover, Three Black Crows, Bearish Harami

**Outputs**
- A table with: **Stock Symbol**, **Pattern Detected**, **Bullish/Bearish**, **When this pattern usually forms**
- A summary listing **bullish swing candidates** and **bearish/avoid** symbols
- CSV export of results

> Notes:
> - TA-Lib returns **+100 / -100 / 0** for most candle functions. Positive values indicate bullish, negative indicate bearish.
> - We apply a simple trend filter to make signals more relevant for swing trading (bullish after a downtrend, bearish after an uptrend).


## 1) Setup

Run the cell below to install dependencies.  
**If TA-Lib fails to install via pip**, you may need system libs:
- macOS (brew): `brew install ta-lib`
- Ubuntu/Debian: `sudo apt-get install -y ta-lib`
- Windows: use prebuilt wheels from Christoph Gohlke or `pip install TA-Lib` in an environment that has the binaries.

After installing system libs, re-run the cell to let `pip install ta-lib` succeed.

In [38]:
import sys, subprocess, importlib, os

def ensure(pkg, pip_name=None):
    try:
        importlib.import_module(pkg)
        print(f"OK: {pkg}")
    except ImportError:
        name = pip_name or pkg
        print(f"Installing {name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", name])

# Core deps
for p in ["yfinance", "pandas", "numpy"]:
    ensure(p)

# TA-Lib (python wrapper). If this fails, install system library and retry.
try:
    import talib as ta
    print("OK: TA-Lib already available")
except Exception as e:
    print("TA-Lib not found or failed to import. Attempting pip install...")
    try:
        ensure("talib", "TA-Lib")
        import talib as ta
        print("OK: TA-Lib installed.")
    except Exception as e2:
        print("\n*** TA-Lib installation failed ***")
        print("Error:", e2)
        print("Please install TA-Lib system library and then rerun this cell.")
        raise

import pandas as pd
import numpy as np
import yfinance as yf

pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 160)

OK: yfinance
OK: pandas
OK: numpy
OK: TA-Lib already available


## 2) Parameters

- **symbols**: your list of tickers (e.g., `['RELIANCE.NS','TCS.NS']`)
- **timeframe**: `'1d'` for Daily or `'4h'` for 4-hour
- **lookback_days**: (Daily only) window to fetch
- **recent_window**: how many latest candles to search for a fresh signal


In [39]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# EDIT THESE
symbols = ['ABB.NS', 'ADANIENSOL.NS', 'ADANIGREEN.NS', 'ADANIPOWER.NS', 'AMBUJACEM.NS', 'DMART.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BANKBARODA.NS', 'BPCL.NS', 'BOSCHLTD.NS', 'BRITANNIA.NS', 'CGPOWER.NS', 'CANBK.NS', 'CHOLAFIN.NS', 'DLF.NS', 'DABUR.NS', 'DIVISLAB.NS', 'GAIL.NS', 'GODREJCP.NS', 'HAVELLS.NS', 'HAL.NS', 'HYUNDAI.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'INDHOTEL.NS', 'IOC.NS', 'IRFC.NS', 'NAUKRI.NS', 'INDIGO.NS', 'JSWENERGY.NS', 'JINDALSTEL.NS', 'LTIM.NS', 'LICI.NS', 'LODHA.NS', 'PIDILITIND.NS', 'PFC.NS', 'PNB.NS', 'RECLTD.NS', 'MOTHERSON.NS', 'SHREECEM.NS', 'SIEMENS.NS', 'SWIGGY.NS', 'TVSMOTOR.NS', 'TATAPOWER.NS', 'TORNTPHARM.NS', 'UNITDSPR.NS', 'VBL.NS', 'VEDL.NS', 'ZYDUSLIFE.NS']

timeframe = "4h"        # '1d' (Daily) or '4h' (4-Hour)
lookback_days = 365      # Only used for Daily
recent_window = 3        # How many last candles to search for a fresh signal
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

assert timeframe in ("1d", "4h"), "timeframe must be '1d' or '4h'"

# yfinance fetch params
if timeframe == "1d":
    period = f"{min(max(lookback_days, 2), 1825)}d"  # cap at 5y for speed
    interval = "1d"
else:
    period = "60d"  # yfinance intraday reliable max
    interval = "4h" 

## 3) Helpers

In [40]:
def fetch_ohlc(ticker: str, period: str, interval: str) -> pd.DataFrame:
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=False, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        # If user passed multiple tickers accidentally, pick this one if present
        try:
            df = df.xs(ticker, axis=1, level=1)
        except Exception:
            pass
    # Standardize column names
    cols = {c:c.title() for c in df.columns}
    df = df.rename(columns=cols)
    df = df.dropna()
    print(df.head())
    return df

def slope(series: pd.Series, window: int = 5) -> float:
    if len(series) < window:
        return 0.0
    y = series.tail(window).values.astype(float)
    x = np.arange(len(y))
    m = np.polyfit(x, y, 1)[0]
    return float(m)

def in_downtrend(df: pd.DataFrame, window: int = 5) -> bool:
    if len(df) < 20:
        return False
    sma20 = df['Close'].rolling(20).mean().iloc[-1]
    return slope(df['Close'], window) < 0 and df['Close'].iloc[-1] < (sma20 if pd.notna(sma20) else df['Close'].iloc[-1])

def in_uptrend(df: pd.DataFrame, window: int = 5) -> bool:
    if len(df) < 20:
        return False
    sma20 = df['Close'].rolling(20).mean().iloc[-1]
    return slope(df['Close'], window) > 0 and df['Close'].iloc[-1] > (sma20 if pd.notna(sma20) else df['Close'].iloc[-1])

## 4) TA-Lib Candlestick Detectors

We use TA-Lib's candle functions, and then apply a simple **trend filter** to emphasize swing relevance:
- **Bullish** patterns are only accepted when the recent trend is **down**.
- **Bearish** patterns are only accepted when the recent trend is **up**.

We look for a non-zero signal in the **last `recent_window` candles**, and take the **most recent** one if multiple fire.


In [41]:
import talib as ta

# Map pattern names to TA-Lib functions and their polarity
# We'll compute all and then check last N bars.
PATTERN_FUNCS = {
    # Bullish
    "Bullish Engulfing": lambda o,h,l,c: ta.CDLENGULFING(o,h,l,c),
    "Morning Star":      lambda o,h,l,c: ta.CDLMORNINGSTAR(o,h,l,c),
    "Hammer":            lambda o,h,l,c: ta.CDLHAMMER(o,h,l,c),
    "Piercing Line":     lambda o,h,l,c: ta.CDLPIERCING(o,h,l,c),
    "Three White Soldiers": lambda o,h,l,c: ta.CDL3WHITESOLDIERS(o,h,l,c),
    "Bullish Harami":    lambda o,h,l,c: ta.CDLHARAMI(o,h,l,c),

    # Bearish
    "Bearish Engulfing": lambda o,h,l,c: ta.CDLENGULFING(o,h,l,c),
    "Evening Star":      lambda o,h,l,c: ta.CDLEVENINGSTAR(o,h,l,c),
    "Shooting Star":     lambda o,h,l,c: ta.CDLSHOOTINGSTAR(o,h,l,c),
    "Dark Cloud Cover":  lambda o,h,l,c: ta.CDLDARKCLOUDCOVER(o,h,l,c),
    "Three Black Crows": lambda o,h,l,c: ta.CDL3BLACKCROWS(o,h,l,c),
    "Bearish Harami":    lambda o,h,l,c: ta.CDLHARAMI(o,h,l,c),
}

PATTERN_SIDE = {
    "Bullish Engulfing": "Bullish",
    "Morning Star": "Bullish",
    "Hammer": "Bullish",
    "Piercing Line": "Bullish",
    "Three White Soldiers": "Bullish",
    "Bullish Harami": "Bullish",
    "Bearish Engulfing": "Bearish",
    "Evening Star": "Bearish",
    "Shooting Star": "Bearish",
    "Dark Cloud Cover": "Bearish",
    "Three Black Crows": "Bearish",
    "Bearish Harami": "Bearish",
}

PATTERN_CONTEXT = {
    "Bullish Engulfing": "Typically near the end of a **downtrend**, signaling a potential **bullish reversal**.",
    "Morning Star": "After a **downtrend**: red candle, small 'star', strong green — classic **reversal**.",
    "Hammer": "At/near the end of a **downtrend**; long lower wick indicates **rejection of lows**.",
    "Piercing Line": "Following a **downtrend**; gap-down open then close above prior midpoint — **reversal**.",
    "Three White Soldiers": "Three strong advances after weakness — **bullish reversal/continuation**.",
    "Bullish Harami": "Small green within a large red after a **downtrend** — **reversal** potential.",
    "Bearish Engulfing": "Usually after an **uptrend**, may indicate **bearish reversal**.",
    "Evening Star": "After an **uptrend**: green, small 'star', strong red — **reversal**.",
    "Shooting Star": "At/near the end of an **uptrend**; long upper wick shows **rejection of highs**.",
    "Dark Cloud Cover": "After an **uptrend**; gap-up then close below prior midpoint — **reversal** potential.",
    "Three Black Crows": "Three strong declines after strength — **bearish reversal/continuation**.",
    "Bearish Harami": "Small red within a large green after an **uptrend** — **reversal** potential.",
}

BULLISH_SET = {"Bullish Engulfing","Morning Star","Hammer","Piercing Line","Three White Soldiers","Bullish Harami"}
BEARISH_SET = {"Bearish Engulfing","Evening Star","Shooting Star","Dark Cloud Cover","Three Black Crows","Bearish Harami"}

def find_latest_signal(df: pd.DataFrame, recent_window: int = 3):
    if len(df) < 30:
        return None  # not enough data for trend filters

    o = df["Open"].astype(float).values
    h = df["High"].astype(float).values
    l = df["Low"].astype(float).values
    c = df["Close"].astype(float).values

    # Precompute trend
    up = in_uptrend(df)
    down = in_downtrend(df)

    best = None  # (pattern_name, index, value)
    for name, fn in PATTERN_FUNCS.items():
        vals = fn(o,h,l,c)  # ndarray of ints (0 or +/-100)
        # Search last `recent_window` candles
        start = max(0, len(vals)-recent_window)
        segment = vals[start:]
        # find last non-zero from the end
        nz_idx = None
        for i in range(len(segment)-1, -1, -1):
            if segment[i] != 0:
                nz_idx = start + i
                break
        if nz_idx is None:
            continue

        v = int(vals[nz_idx])
        side = PATTERN_SIDE[name]

        # Enforce trend filter
        if side == "Bullish" and not down:
            continue
        if side == "Bearish" and not up:
            continue

        # store the most recent by index (higher == more recent)
        if (best is None) or (nz_idx > best[1]):
            best = (name, nz_idx, v)

    if best is None:
        return None
    name, idx, v = best
    side = PATTERN_SIDE[name]

    # Handle Harami sign: TA-Lib returns + for bullish, - for bearish on same function
    if name == "Bullish Harami" and v < 0:
        return None
    if name == "Bearish Harami" and v > 0:
        return None

    return {
        "pattern": name,
        "side": side,
        "index": idx,
        "strength": abs(v),
        "context": PATTERN_CONTEXT[name],
    }

## 5) Scan Symbols

In [42]:
def scan_symbol(ticker: str) -> dict:
    try:
        df = fetch_ohlc(ticker, period=period, interval=interval)
        if df.empty:
            return dict(symbol=ticker, pattern="No clear setup", side="", context="No data")
        sig = find_latest_signal(df, recent_window=recent_window)
        if sig is None:
            return dict(symbol=ticker, pattern="No clear setup", side="", context="—")
        return dict(symbol=ticker, pattern=sig["pattern"], side=sig["side"], context=sig["context"])
    except Exception as e:
        return dict(symbol=ticker, pattern="Error", side="", context=str(e))

rows = [scan_symbol(t) for t in symbols]
results = pd.DataFrame(rows, columns=["symbol","pattern","side","context"])
results.rename(columns={
    "symbol":"Stock Symbol",
    "pattern":"Pattern Detected",
    "side":"Bullish/Bearish",
    "context":"When this pattern usually forms"
}, inplace=True)

results

Price                      Adj Close   Close    High     Low    Open  Volume
Datetime                                                                    
2025-06-13 03:45:00+00:00     5950.0  5950.0  5980.0  5892.0  5898.5   60329
2025-06-13 07:45:00+00:00     6001.0  6001.0  6017.0  5942.5  5950.0  102655
2025-06-16 03:45:00+00:00     6020.0  6020.0  6041.5  5941.0  5996.5   54552
2025-06-16 07:45:00+00:00     6010.0  6010.0  6030.0  6004.5  6020.0   46940
2025-06-17 03:45:00+00:00     6037.0  6037.0  6084.0  6014.0  6025.0   72517
Price                       Adj Close       Close        High         Low        Open   Volume
Datetime                                                                                      
2025-06-13 03:45:00+00:00  856.650024  856.650024  871.549988  854.750000  860.250000  2079166
2025-06-13 07:45:00+00:00  863.000000  863.000000  867.500000  856.750000  857.150024  1664792
2025-06-16 03:45:00+00:00  866.700012  866.700012  869.000000  849.500000  858.25

,Stock Symbol,Pattern Detected,Bullish/Bearish,When this pattern usually forms
0,ABB.NS,No clear setup,,—
1,ADANIENSOL.NS,No clear setup,,—
2,ADANIGREEN.NS,No clear setup,,—
3,ADANIPOWER.NS,No clear setup,,—
4,AMBUJACEM.NS,No clear setup,,—
5,DMART.NS,No clear setup,,—
6,BAJAJHLDNG.NS,No clear setup,,—
7,BAJAJHFL.NS,Bullish Engulfing,Bullish,"Typically near the end of a **downtrend**, sig..."
8,BANKBARODA.NS,No clear setup,,—
9,BPCL.NS,No clear setup,,—


## 6) Export & Summary

In [43]:
# Save results
out_csv = f"talib_candlestick_scan_{timeframe}.csv"
results.to_csv(out_csv, index=False)
print(f"Saved: {out_csv}")

# Summaries
bullish = results[results["Bullish/Bearish"]=="Bullish"]["Stock Symbol"].tolist()
bearish = results[results["Bullish/Bearish"]=="Bearish"]["Stock Symbol"].tolist()

print("\nSummary:")
print("• **Bullish swing candidates**:", ", ".join(bullish) if bullish else "None")
print("• **Bearish / avoid for now**:", ", ".join(bearish) if bearish else "None")

Saved: talib_candlestick_scan_4h.csv

Summary:
• **Bullish swing candidates**: BAJAJHFL.NS, DIVISLAB.NS, ICICIGI.NS, NAUKRI.NS, LTIM.NS, TORNTPHARM.NS, UNITDSPR.NS
• **Bearish / avoid for now**: SHREECEM.NS, ZYDUSLIFE.NS


### Notes & Next Steps
- Consider confirming signals with **volume**, **RSI/MACD**, and **support/resistance**.
- Use **risk management**: stops below swing lows (bullish) or above swing highs (bearish), position sizing, take-profits.
- You can expand this notebook to add charts, alerts, or integrate with your backtesting pipeline.
